<a href="https://colab.research.google.com/github/UF-CIS6930-NLP-Semantalists/semantic-shift-communities/blob/main/CIS6930_BERT_Pre_training_liberal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-pf1lad59
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-pf1lad59
  Resolved https://github.com/huggingface/transformers to commit fb3aa06cb673d0e2774a2924747d3290135c09cc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=6928727 sha256=0fa34e15dab67351c61f5e510cd2a6bb6ddc6430709bef2746e4b26e6ed17a44
  Stored in directory: /tmp/pip-ephem-wheel-cache-1g1brmco/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190

In [ ]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "bert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> BERT number of parameters: {round(bert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> BERT number of parameters: 110M'
'>>> BERT number of parameters: 110M'


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import torch

text = "This is a [MASK] day."
inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a beautiful day.'
'>>> This is a good day.'
'>>> This is a new day.'
'>>> This is a great day.'
'>>> This is a big day.'


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 18.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset

# dataset = pd.read_csv("/content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project/conservative_base_sample.csv", encoding='utf8')
# full_text = dataset.body
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project/Conservative_bert/liberal_base_sample.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets import Dataset

df = pd.read_csv("/content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project/Conservative_bert/liberal_base_sample.csv", encoding="utf8")
df = pd.DataFrame(df)
df = df.dropna()
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['body', 'controversiality', 'permalink', 'score', 'subreddit', 'id', 'timestamp', 'preprocess_body', 'num_sentences', 'numWords', '__index_level_0__'],
    num_rows: 194936
})

In [ ]:
dataset[9526]

{'body': 'ðŸ˜˜ðŸ˜˜',
 'controversiality': 0,
 'permalink': '/r/Liberal/comments/gfwrxq/barrs_corrupt_decision_points_to_trumps_moral_rot/fpz70j6/',
 'score': 1,
 'subreddit': 'Liberal',
 'id': 'fpz70j6',
 'timestamp': 1589014423,
 'preprocess_body': "['ðÿ˜˜ðÿ˜˜']",
 'num_sentences': 1,
 'numWords': 1,
 '__index_level_0__': 9526}

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["body"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
        # result["word_ids"] = result.word_ids()
    return result


# Use batched=True to activate fast multithreading!
# tokenize_function(dataset['train'][9526])
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=['body','controversiality', 'permalink', 'score', 'subreddit', 'id', 'timestamp', 'preprocess_body', 'num_sentences', 'numWords', '__index_level_0__']
)
tokenized_datasets

Map:   0%|          | 0/194936 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 194936
})

In [ ]:
print(tokenized_datasets[0])

{'input_ids': [101, 8840, 2140, 2043, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1], 'word_ids': [None, 0, 0, 1, None]}


In [ ]:
tokenizer.model_max_length

512

In [ ]:
chunk_size = 128

In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets[:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Post {idx} length: {len(sample)}'")

'>>> Post 0 length: 5'
'>>> Post 1 length: 114'
'>>> Post 2 length: 11'


In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 130'


In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 2'


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/194936 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 76408
})

In [ ]:
tokenizer.decode(lm_datasets[0]["input_ids"])

"[CLS] lol when [SEP] [CLS] no reference materials in this article on the daily mail. which is already questionably news. and then they talk about how he helped the shiites during the us invasion of iraq. that's not terrorism. the french supporting us during our revolution wasn't terrorism. it was war, picking a side and supporting it because that side supports your values. i think those values, which assign very closely with evangelicals, but i digress, are deplorable. but it's still not terrorism to help a country fight off an invading force. [SEP] [CLS] biased news source is biased without any references"

In [ ]:
tokenizer.decode(lm_datasets[0]["labels"])

"[CLS] lol when [SEP] [CLS] no reference materials in this article on the daily mail. which is already questionably news. and then they talk about how he helped the shiites during the us invasion of iraq. that's not terrorism. the french supporting us during our revolution wasn't terrorism. it was war, picking a side and supporting it because that side supports your values. i think those values, which assign very closely with evangelicals, but i digress, are deplorable. but it's still not terrorism to help a country fight off an invading force. [SEP] [CLS] biased news source is biased without any references"

In [ ]:
# masking for training
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] lol when [SEP] [CLS] no reference materials in this article on [MASK] daily mail. which is already question [MASK] news. [MASK] then [MASK] talk [MASK]gram he helped the shiites during the us invasion of iraq. [MASK] [MASK] s not terrorism. the french supporting us during our revolution wasn't terrorism. it was main, picking [MASK] side and supporting it because that [MASK] supports your [MASK]. i think those values, which [MASK] very closely with evangelicals, but i digress, are deplorable. but it's still not terrorism [MASK] [MASK] a country fight [MASK] seychellesasco force [MASK] [SEP] [CLS] biased news source [MASK] biased without any references'

'>>> . [SEP] [CLS] observed terrorists " [SEP] [CLS] careful now, talk like that will get you banned [MASK] [SEP] [CLS] you were downvoted by people who want to suck our wanna be [MASK] dictatora€ [MASK] [MASK] ass [SEP] [CLS] [MASK]'s something kinship consider. we [MASK] [MASK] have to see [MASK] it plays out. [SEP] [CLS] h

In [ ]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [ ]:
samples = [lm_datasets[i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] lol when [SEP] [CLS] no reference materials in this article on the daily [MASK]. which is [MASK] questionably news. and [MASK] they talk about how he helped the shiites during the us invasion [MASK] iraq. that [MASK] s not [MASK]. the [MASK] supporting us during [MASK] revolution [MASK]'t terrorism. [MASK] [MASK] war, [MASK] a side and supporting it because [MASK] side supports your values. i think those [MASK], [MASK] [MASK] very [MASK] with evangelicals, but i digress [MASK] are deplorable [MASK] but it'[MASK] still not terrorism [MASK] help a [MASK] [MASK] off an invading force. [SEP] [CLS] biased news source is biased without [MASK] references'

'>>> . [SEP] [CLS] " terrorists " [SEP] [CLS] careful now, talk like that [MASK] get you banned. [SEP] [CLS] [MASK] were downvoted by people who [MASK] [MASK] suck our wanna be orange dictatora€™s [MASK] [SEP] [CLS] [MASK] [MASK] s something to [MASK]. we [MASK] [MASK] have to [MASK] how it plays out. [SEP] [CLS] he definitely [

In [ ]:
train_size = 75_000
test_size = 1408

downsampled_dataset = lm_datasets.train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 75000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1408
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-reddit_liberal",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    remove_unused_columns=False,
    logging_steps=logging_steps,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=whole_word_masking_data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/ChathuriJ/bert-base-uncased-finetuned-reddit_liberal into local empty directory.


In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 89.37


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.983000,2.618840
2,2.719900,2.540663
3,2.652900,2.537279


TrainOutput(global_step=3516, training_loss=2.7851325335898633, metrics={'train_runtime': 2291.6474, 'train_samples_per_second': 98.183, 'train_steps_per_second': 1.534, 'total_flos': 1.480527072e+16, 'train_loss': 2.7851325335898633, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 12.81


In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file runs/Apr15_04-00-39_6c526a6e9e3c/events.out.tfevents.1681531307.6c526a6e9e3c.1079.0:   0%|        …

Upload file runs/Apr15_04-00-39_6c526a6e9e3c/events.out.tfevents.1681533991.6c526a6e9e3c.1079.2:   0%|        …

To https://huggingface.co/ChathuriJ/bert-base-uncased-finetuned-reddit_liberal
   f3b4edc..cabf71c  main -> main

   f3b4edc..cabf71c  main -> main

To https://huggingface.co/ChathuriJ/bert-base-uncased-finetuned-reddit_liberal
   cabf71c..e0f1992  main -> main

   cabf71c..e0f1992  main -> main



'https://huggingface.co/ChathuriJ/bert-base-uncased-finetuned-reddit_liberal/commit/cabf71cfb3a1f962c88acf2ec5fa2ce043419ebe'

# Misc

In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-9k9u4q6q
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-9k9u4q6q
  Resolved https://github.com/huggingface/transformers to commit 410b61ad7e8f69113a86d0003190e3c392c7c39a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=6928596 sha256=db78c3b9af7b93796e88200a3622d839988452381779b5f92237d05653b

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer.is_fast

True

In [ ]:
def batch_iterator():
  for i in range(0, len(data), batch_size):
    yield data[i : i + batch_size]

In [ ]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=52000, max_length=600)

In [ ]:
data[0]

"100% agree and I'm a leftist. Most leftists feel the same, and we could make serious progress if we worked together. The question is: what values and institutions replace those which exist in the status quo?\n\nWe'd likely disagree there, but I find many conservatives much easier to talk to about these fundamental issues for this exact reason."

In [ ]:
new_tokenizer(data[0])

{'input_ids': [2, 2013, 9, 1693, 1068, 49, 11, 53, 41, 2438, 18, 1392, 2666, 1759, 1049, 1468, 16, 1068, 1109, 1405, 1409, 2818, 2678, 1157, 1109, 3128, 3107, 18, 1049, 2056, 1077, 30, 1177, 3562, 1068, 6309, 4975, 1429, 1404, 2373, 1067, 1049, 3773, 6095, 33, 1109, 11, 44, 2133, 2434, 1210, 16, 1136, 49, 1823, 1502, 1910, 1427, 3906, 1060, 2441, 1060, 1199, 1447, 6752, 2570, 1094, 1114, 1911, 1625, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
max(data.apply(lambda x: len(new_tokenizer(x)['input_ids'])))

AttributeError: ignored

In [ ]:
%cd /content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project/

/content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project


In [ ]:
!mkdir Conservative_bert
%cd Conservative_bert/
new_tokenizer.save_pretrained("conservative_bert_tok")

/content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project/Conservative_bert


('conservative_bert_tok/tokenizer_config.json',
 'conservative_bert_tok/special_tokens_map.json',
 'conservative_bert_tok/vocab.txt',
 'conservative_bert_tok/added_tokens.json',
 'conservative_bert_tok/tokenizer.json')

In [ ]:
data = dataset.body
tokenized = data.apply((lambda x: new_tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
tokenized

0         [2, 2013, 9, 1693, 1068, 49, 11, 53, 41, 2438,...
1         [2, 1366, 30, 19, 19, 1529, 18, 1701, 18, 1133...
2         [2, 6, 1109, 1552, 1049, 6528, 17, 7656, 17, 8...
3         [2, 1068, 1348, 1104, 155, 59, 2430, 9727, 108...
4         [2, 49, 6878, 1114, 1092, 2635, 1068, 1329, 49...
                                ...                        
194939    [2, 36, 10, 1295, 31, 4107, 4879, 9, 1069, 115...
194940    [2, 2123, 18527, 6645, 3748, 1747, 26896, 3483...
194941    [2, 1257, 1221, 1900, 1213, 1069, 1221, 41, 36...
194942    [2, 1724, 1886, 4672, 1804, 1114, 1307, 1130, ...
194943    [2, 2573, 49, 2236, 3628, 1179, 2637, 1449, 10...
Name: body, Length: 194944, dtype: object

In [ ]:
print(new_tokenizer.tokenize(data[0]))

['100', '%', 'agree', 'and', 'i', "'", 'm', 'a', 'leftist', '.', 'most', 'leftists', 'feel', 'the', 'same', ',', 'and', 'we', 'could', 'make', 'serious', 'progress', 'if', 'we', 'worked', 'together', '.', 'the', 'question', 'is', ':', 'what', 'values', 'and', 'institutions', 'replace', 'those', 'which', 'exist', 'in', 'the', 'status', 'quo', '?', 'we', "'", 'd', 'likely', 'disagree', 'there', ',', 'but', 'i', 'find', 'many', 'conservatives', 'much', 'easier', 'to', 'talk', 'to', 'about', 'these', 'fundamental', 'issues', 'for', 'this', 'exact', 'reason', '.']


In [ ]:
new_tokenizer.decode(new_tokenizer.encode(data[0]))

"[CLS] 100 % agree and i'm a leftist. most leftists feel the same, and we could make serious progress if we worked together. the question is : what values and institutions replace those which exist in the status quo? we'd likely disagree there, but i find many conservatives much easier to talk to about these fundamental issues for this exact reason. [SEP]"

In [ ]:
# Check that we have a GPU
!nvidia-smi

Thu Apr 13 21:24:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import BertConfig


config = BertConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)